# Sprint 2: Image Restoration Engine

**Project:** Road Defect Detection System (PROSIT 1)  
**Team Members:**
- Naa Lamle Boye
- Thomas Kojo Quarshie
- Chelsea Owusu
- Elijah Boateng

**Date:** 2024

## Purpose

This notebook implements image restoration techniques to improve image quality for road defect detection.

**What this notebook achieves:**
- Removes motion blur from images using deconvolution techniques (Wiener filter, Richardson-Lucy)
- Stabilizes image sequences to remove camera shake
- Removes shadows using Multi-Scale Retinex algorithm
- Provides a complete restoration pipeline combining multiple techniques

**Key Features:**
- Motion blur removal (Wiener deconvolution, Richardson-Lucy deconvolution)
- Image stabilization (optical flow-based)
- Shadow removal (Retinex-based with CLAHE enhancement)

## Step 1: Import Required Libraries

In [1]:
import numpy as np
import cv2
from scipy import ndimage

## Step 2: Motion Blur Removal

Implement deblurring functions using Wiener deconvolution and Richardson-Lucy deconvolution algorithms.

In [2]:
def deblur_image(image, method='wiener', kernel_size=5):
    """
    Remove motion blur from image using deconvolution.
    
    Args:
        image: Input blurred image (BGR or grayscale)
        method: 'wiener' or 'richardson_lucy'
        kernel_size: Size of blur kernel (assumes horizontal motion blur)
    
    Returns:
        np.ndarray: Deblurred image
    """
    if method == 'wiener':
        # Create motion blur kernel (horizontal motion)
        kernel = np.zeros((kernel_size, kernel_size))
        kernel[int((kernel_size-1)/2), :] = np.ones(kernel_size)
        kernel = kernel / kernel_size
        
        # Wiener deconvolution in frequency domain
        kernel_fft = np.fft.fft2(kernel, s=image.shape[:2])
        image_fft = np.fft.fft2(image, axes=(0, 1))
        
        # Simple Wiener filter
        # SNR (Signal-to-Noise Ratio) estimate - can be tuned based on image quality
        snr = 100
        wiener = np.conj(kernel_fft) / (np.abs(kernel_fft)**2 + 1/snr)
        
        # Apply filter and convert back to spatial domain
        if len(image.shape) == 3:
            restored = np.zeros_like(image)
            for i in range(3):
                restored[:, :, i] = np.real(np.fft.ifft2(image_fft[:, :, i] * wiener))
        else:
            restored = np.real(np.fft.ifft2(image_fft * wiener))
        
        restored = np.clip(restored, 0, 255).astype(np.uint8)
        return restored
    
    elif method == 'richardson_lucy':
        # Richardson-Lucy deconvolution (iterative method)
        kernel = np.zeros((kernel_size, kernel_size))
        kernel[int((kernel_size-1)/2), :] = np.ones(kernel_size)
        kernel = kernel / kernel_size
        
        if len(image.shape) == 3:
            restored = np.zeros_like(image, dtype=np.float64)
            for i in range(3):
                restored[:, :, i] = ndimage.richardson_lucy(
                    image[:, :, i].astype(np.float64), kernel, num_iter=10
                )
        else:
            restored = ndimage.richardson_lucy(
                image.astype(np.float64), kernel, num_iter=10
            )
        
        restored = np.clip(restored, 0, 255).astype(np.uint8)
        return restored
    
    return image

## Step 3: Image Stabilization

Stabilize sequence of images to remove camera shake using optical flow.

In [3]:
def stabilize_image(image_sequence, method='optical_flow'):
    """
    Stabilize sequence of images to remove camera shake.
    
    Args:
        image_sequence: List of images or video frames
        method: 'optical_flow' or 'feature_matching'
    
    Returns:
        list: Stabilized images
    """
    if len(image_sequence) < 2:
        return image_sequence
    
    stabilized = [image_sequence[0]]
    
    if method == 'optical_flow':
        prev_gray = cv2.cvtColor(image_sequence[0], cv2.COLOR_BGR2GRAY)
        
        for i in range(1, len(image_sequence)):
            curr_gray = cv2.cvtColor(image_sequence[i], cv2.COLOR_BGR2GRAY)
            
            # Calculate optical flow between consecutive frames
            # Farneback method computes dense optical flow
            flow = cv2.calcOpticalFlowFarneback(
                prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0
            )
            
            # Calculate average translation (camera movement)
            dx = np.mean(flow[:, :, 0])
            dy = np.mean(flow[:, :, 1])
            
            # Apply inverse translation to stabilize
            M = np.float32([[1, 0, -dx], [0, 1, -dy]])
            h, w = image_sequence[i].shape[:2]
            stabilized_frame = cv2.warpAffine(image_sequence[i], M, (w, h))
            stabilized.append(stabilized_frame)
            
            prev_gray = curr_gray
    
    return stabilized

## Step 4: Shadow Removal

Remove shadows using Multi-Scale Retinex algorithm with CLAHE enhancement.

In [4]:
def remove_shadows_retinex(image, sigma_list=[15, 80, 250]):
    """
    Remove shadows using Multi-Scale Retinex algorithm.
    
    The Retinex algorithm models the image as: I(x,y) = R(x,y) * L(x,y)
    where R is reflectance (intrinsic color) and L is illumination (lighting).
    By removing the illumination component, we can reduce shadows.
    
    Args:
        image: Input image with shadows (BGR or grayscale)
        sigma_list: List of Gaussian blur sigma values for multi-scale processing
                   Different scales capture shadows of different sizes
    
    Returns:
        np.ndarray: Shadow-removed image
    """
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    
    # Convert to float and normalize to [0, 1]
    image_float = image.astype(np.float64) / 255.0
    log_image = np.log(image_float + 1e-10)  # Add small epsilon to avoid log(0)
    
    retinex = np.zeros_like(image_float)
    
    # Multi-scale Retinex: process at different scales
    for sigma in sigma_list:
        # Gaussian blur approximates the illumination component
        blurred = cv2.GaussianBlur(image_float, (0, 0), sigma)
        log_blurred = np.log(blurred + 1e-10)
        
        # Retinex: log(image) - log(blurred) = log(reflectance)
        retinex += (log_image - log_blurred)
    
    # Average across scales
    retinex = retinex / len(sigma_list)
    
    # Normalize each channel to [0, 1] and enhance contrast
    for i in range(3):
        channel = retinex[:, :, i]
        channel_min = np.min(channel)
        channel_max = np.max(channel)
        if channel_max > channel_min:
            retinex[:, :, i] = (channel - channel_min) / (channel_max - channel_min)
    
    # Convert back to uint8
    result = (retinex * 255).astype(np.uint8)
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) for better contrast
    lab = cv2.cvtColor(result, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    result = cv2.merge([l, a, b])
    result = cv2.cvtColor(result, cv2.COLOR_LAB2BGR)
    
    return result

## Step 5: Complete Restoration Pipeline

Combine all restoration techniques into a single pipeline function.

In [5]:
def restore_image(image, deblur=True, remove_shadows=True, deblur_method='wiener'):
    """
    Complete restoration pipeline combining multiple techniques.
    
    Args:
        image: Input degraded image
        deblur: Whether to apply deblurring
        remove_shadows: Whether to remove shadows
        deblur_method: 'wiener' or 'richardson_lucy'
    
    Returns:
        np.ndarray: Restored image
    """
    result = image.copy()
    
    if deblur:
        result = deblur_image(result, method=deblur_method)
    
    if remove_shadows:
        result = remove_shadows_retinex(result)
    
    return result

## Step 6: Test Restoration Functions

Test the restoration functions with a sample image.

In [6]:
# Create test image with blur and shadows
test_image = np.ones((400, 600, 3), dtype=np.uint8) * 150

# Add some structure
cv2.rectangle(test_image, (100, 100), (300, 200), (200, 200, 200), -1)
cv2.circle(test_image, (400, 250), 80, (100, 100, 100), -1)

# Add simulated shadow
cv2.ellipse(test_image, (300, 150), (150, 100), 45, 0, 360, (50, 50, 50), -1)

# Test deblurring
deblurred = deblur_image(test_image, method='wiener')
print(f"Deblurred image shape: {deblurred.shape}")

# Test shadow removal
shadow_removed = remove_shadows_retinex(test_image)
print(f"Shadow-removed image shape: {shadow_removed.shape}")

# Test full restoration
restored = restore_image(test_image, deblur=True, remove_shadows=True)
print(f"Restored image shape: {restored.shape}")

# Save results
cv2.imwrite("test_deblur_output.png", deblurred)
cv2.imwrite("test_shadow_removal_output.png", shadow_removed)
cv2.imwrite("test_restoration_output.png", restored)
print("Test outputs saved.")

Deblurred image shape: (400, 600, 3)


Shadow-removed image shape: (400, 600, 3)


Restored image shape: (400, 600, 3)
Test outputs saved.
